# MNIST with TensorFlow

대표적인 딥 러닝 프레임워크인 TensorFlow로 MNIST classification을 구현해 봅시다!

* MNIST?

  MNIST는 28px×28px의 숫자 이미지들과 여기 해당하는 레이블들로 구성된 데이터베이스입니다.
  
  ![MNIST Example](http://solarisailab.com/wp-content/uploads/2016/05/mnist_2.gif)
  
* 오늘 해볼 것

  오늘은 이 이미지가 들어오면 이미지에 적힌 숫자가 무엇인지를 판단하는 프로그램을 TensorFlow로 짜 봅시다. Lecture 2~3에서 등장한 Softmax Classifier를 사용하는 것으로 하고, TensorFlow 코드가 어떻게 생겼는지 보도록 합시다.
  

일단 MNIST 데이터셋을 다운로드합니다. TensorFlow가 있다면 `tensorflow.examples.tutorials.mnist` 패키지에 이미 데이터셋이 있습니다.
MNIST의 레이블은 0~9의 값이지만, 이것은 연속된 숫자가 아닌 카테고리 값입니다. 따라서 One-Hot Encoding이 필요합니다. 5는 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 0은 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0] 식으로 됩니다. 간단하게 말하면 문자를 숫자로 변형해주는 방법들 중 하나라고 보시면 되겠습니다.

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


그리고 TensorFlow 라이브러리를 가져옵니다.

In [22]:
import tensorflow as tf

이제 변수들을 설정합니다.

__Remark__: W = weight, x = 이미지의 각 픽셀 값, b = bias라고 할 때 결과 y = softmax(Wx + b) 이었습니다.

* `tf.placeholder`: 입력 값이 들어올 수 있는 변수. 여기서 None은 행의 수가 한정되지 않는다는 뜻입니다. tf.placeholder(dtype,shape) 
* `W`는 [784, 10]의 행태를 가지는데, 이것은 784차원의 이미지 벡터를 곱해, 10차원(인코딩된 0~9)의 결과를 내기 위한 것입니다.
* `b`는 결과에 더하기 위해 10차원이 됩니다.
* `tf.Variable`: 변수. 실제로 값이 존재하는 변수는 아니고, Computational Graph의 일부입니다. 후술할 `sess.run()` 이후에 값이 결정됩니다.
  * 아래의 `y = tf.nn.softmax(tf.matmul(x, W) + b)`의 경우 코드를 실행하는 순간에 소프트맥스를 돌리는 건 아니고, **y를 계산하려면 softmax(Wx + b)를 계산해야 한다**는 정보만을 그래프 형태로 저장합니다. 실제 계산은 나중에 됩니다.
* `tf.matmul`: 행렬 곱셈 계산 함수.
* `tf.nn.softmax`: 미리 구현되어 있는 Softmax classifier.

In [23]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

이제 Cross-Entropy cost function을 설정합니다. cross-entropy를 간단히 설명하자면 모델의 예측값이 실제 참값을 설명하는데 얼마나 비효율적인지를 나타내는 함수입니다. 따라서, cross-entropy가 낮을수록 좋은 모델이고, 그렇기 때문에 이 값을 낮추는 쪽으로 함수를 진행시킵니다.

__Remark__: H<sub>y'</sub>(y) = -Σ<sub>i</sub> y<sub>i</sub>' log(y<sub>i</sub>)

* `y_`에 학습용 정답이 채워집니다.
* `reduction_indices[1]`은 y_ * tf.log(y)의 형태가 1 x 10의 행렬인데 이것을 1 x 1로 만들어 모든 값의 평균을 내기 쉽게 만들어주는 역할을 합니다.'
* `tf.zeros`는 0으로 된 텐서를 생성합니다.
* `tf.reduce_sum`: 어떤 축을 기준으로 텐서의 모든 값을 더함 (Σ)
* `tf.reduce_mean`: 어떤 축을 기준으로 텐서의 모든 값의 평균
* `tf.train.GradientDescentOptimizer`: 미리 구현되어 있는 Gradient Descent Optimizer.미분을 통해 최저 비용을 향해 진행하도록 만드는 핵심 함수. 또한 minimize 함수는 글자 그대로 최소 비용을 찾아주는 함수라고 생각하면 됩니다. 그러나, 정확하게는 gradient descent 알고리즘에서 gradients를 계산해서 변수에 적용하는 일을 동시에 하는 함수입니다. W와 b를 적절하게 계산해서 변경하는 역할을 하는데, 그 진행 방향이 cost가 작아지는 쪽이라는 뜻입니다.

In [24]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(
    -tf.reduce_sum(y_ * tf.log(y),
    reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

그리고 생성한 값들로 학습시킵니다.
* `tf.Session`: TensorFlow 연산들을 실제로 실행하는 세션은 그래프를 인자로 받아서 실행을 해주는 일종의 러너(Runner)라고 생각하면 됩니다.
* `mnist.train.next_batch`: `train` 데이터셋에서 데이터 n개 가져오기
* `feed_dict`에 의해서 정해진 피드 데이타를 하나씩 읽어서 실행합니다.
 
※ 랜덤한 작은 배치로 학습 하는 것을 스토캐스틱 학습이라고 합니다. 이것이 비용이 쌀 뿐만아니라 결과도 비슷하다고 합니다.

x가 이미지 픽셀, y가 라벨(이미지에 적혀있는 숫자)입니다.

In [25]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(500)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

정확도를 평가해 봅시다!

* `tf.equal`에서는 예측 값과 정답이 같으면 True 아니면 False 값이 반환되는데, 이것을 float형으로 바꾸고 평균을 계산해 정확도를 구합니다. 정확도는 학습 데이터가 아닌 테스트 데이터를 사용해야합니다.
* 이제 계산된 레이블들 중 가장 점수가 높은 것을 선택합니다. `tf.argmax`함수는 텐서 내의 지정된 축에서 가장 높은 값의 인덱스를 반환합니다.

In [26]:
correct_prediction = tf.equal(
    tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(
    tf.cast(correct_prediction, tf.float32))
print(sess.run(
    accuracy, 
    feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9203
